In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso



In [127]:
# grab data -- change path for your own file
df1 = pd.read_csv("/Users/ethanboroditsky/ML_Focus/inequality_class.csv")

# print first few rows -- could also use df.head()
print (df1.iloc[:3])
# and get summary stats on variables
print (df1.describe())

print(type(df1))

         country  federalism_gt  id region_wb  gdp  statehiste1500_02n  \
0       Abkhazia            NaN NaN       NaN  NaN                 NaN   
1    Afghanistan            0.0 NaN       NaN  NaN            0.422657   
2  Aland Islands            NaN NaN       NaN  NaN                 NaN   

   origtime2  legor_so      eleva  avg_temp    ...      v1_a_lum00pc  \
0        NaN         0        NaN       NaN    ...               NaN   
1    40000.0         0  1836.7983      13.8    ...          0.732304   
2        NaN         0        NaN       NaN    ...               NaN   

   eur_pct_homog   mal_max  tech1500_ext  gini_disp_ext  sd_index_ext  \
0            NaN       NaN           NaN            NaN           NaN   
1           50.0 -0.133697         0.617      31.700001     -0.564311   
2            NaN       NaN           NaN            NaN           NaN   

     rugged       soil  tropical  adj_index  
0       NaN        NaN       NaN        NaN  
1  2.518146  27.848703       

In [128]:
print(type(df1))
df1 = df1.drop(columns=['id', 'region_wb'] , axis=1)

<class 'pandas.core.frame.DataFrame'>


In [129]:
count_row = df1.shape[0]# Gives number of rows
count_col = df1.shape[1] #gives number of columns/variables
limitPer = count_row * .10
df2 = df1.dropna(thresh=limitPer) #gets rid of rows w/ countries w/ not enough data filled in
print (df2.shape)

print(df2.head())
print(df2.tail())
print(df2.describe())


(192, 56)
       country  federalism_gt  gdp  statehiste1500_02n  origtime2  legor_so  \
1  Afghanistan            0.0  NaN            0.422657    40000.0         0   
3      Albania            0.0  NaN            0.374967    45000.0         1   
4      Algeria            0.0  NaN            0.405222    40000.0         0   
6      Andorra            0.0  NaN                 NaN        NaN         0   
7       Angola            0.0  NaN            0.050186   135000.0         0   

        eleva  avg_temp  al_ethnic  al_language    ...      v1_a_lum00pc  \
1  1836.79830      13.8   0.769345     0.614146    ...          0.732304   
3   725.77765      12.5   0.220426     0.039925    ...          0.037976   
4   585.31787      23.4   0.339400     0.442662    ...          0.892659   
6  1996.00000       8.4   0.713946     0.684785    ...               NaN   
7  1099.46630      21.6   0.786720     0.787019    ...          0.815747   

   eur_pct_homog   mal_max  tech1500_ext  gini_disp_ext  s

In [130]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [131]:
df2 = swap_columns(df2, "gini_disp_ext", "adj_index")


In [132]:
headers = list(df2)
headers.remove("country")

In [133]:
# create set of variables to use / exclude Y
vars = headers

In [134]:
x = df2.loc[:, vars].values

# also create Y while we're at it for use later on in regressions
y = df2.loc[:, 'gini_disp_ext'].values
y = df2.reindex

print(x)
print("\n\n")
print(y)

print("\n\n")

print(df2.head())
print(df2.tail())


[[0.0000000e+00           nan           nan ... 2.5181463e+00
  2.7848703e+01 0.0000000e+00]
 [0.0000000e+00           nan           nan ... 3.4270582e+00
  6.8088409e+01 0.0000000e+00]
 [0.0000000e+00           nan           nan ... 5.0975060e-01
  9.5900192e+00 0.0000000e+00]
 ...
 [0.0000000e+00           nan           nan ... 2.3231130e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00           nan 2.6197214e+03 ... 5.3258377e-01
  2.3114420e+01 1.7132454e+01]
 [0.0000000e+00           nan           nan ... 1.1943413e+00
  5.4876743e+01 8.4007568e+00]]



[31.700001 38.900002 35.5      30.700001 49.799999 47.099998 42.099998
 36.700001 32.5      27.799999 29.299999 43.799999 47.599998 34.700001
 43.700001 24.299999 26.5      50.700001 45.799999 41.200001 49.5
 39.299999 58.700001 49.200001 30.780001 33.5      44.099998 38.799999
 49.900002 36.400002 44.900002 31.200001 54.5      42.700001 47.799999
 42.700001 54.299999 55.400002 45.299999 55.200001 27.5      43.169998
 29.700001 2

/Users/ethanboroditsky/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [135]:
df2.shape
df2 = df2.drop(columns=['country'] , axis=1)
df2.shape
df2.head()

,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,al_religion,...,v1_a_lum00pc,eur_pct_homog,mal_max,tech1500_ext,adj_index,sd_index_ext,rugged,soil,tropical,gini_disp_ext
1,0.0,NaN,0.422657,40000.0,0,1836.79830,13.8,0.769345,0.614146,0.271684,...,0.732304,50.000000,-0.133697,0.617,1.405038,-0.564311,2.518146,27.848703,0.000000,31.700001
3,0.0,NaN,0.374967,45000.0,1,725.77765,12.5,0.220426,0.039925,0.471852,...,0.037976,42.840736,-0.744265,0.825,1.451480,-1.452140,3.427058,68.088409,0.000000,38.900002
4,0.0,NaN,0.405222,40000.0,0,585.31787,23.4,0.339400,0.442662,0.009128,...,0.892659,49.142891,-0.286520,0.783,0.422037,-1.716895,0.509751,9.590019,0.000000,35.500000
6,0.0,NaN,NaN,NaN,0,1996.00000,8.4,0.713946,0.684785,0.232568,...,NaN,43.490902,NaN,1.000,1.714573,-1.138500,5.717271,0.000000,0.000000,30.700001
7,0.0,NaN,0.050186,135000.0,0,1099.46630,21.6,0.786720,0.787019,0.627644,...,0.815747,48.803310,1.609371,0.167,-2.669100,-0.101569,0.858216,26.676142,44.346245,49.799999


In [178]:
#DO NOT RUN THIS CELL
# normalize x
dff = df2.fillna(df2.mean())
x_norm = StandardScaler().fit_transform(dff.loc[:, geo])

pca = PCA(n_components=1).fit(x_norm)
new_df = pca.transform(x_norm)
var_exp = pca.explained_variance_ratio_

#important variables: gdp, federalism_gt, origtime2, adj_index, 

#duplicate variables

# try pca -- imagine theory suggests you look for a 1d latent representation for x2,x3,x7
dim = 1
geo = ['eleva', 'avg_temp', 'frstdays', "lp_lat_abst_fill"]
agro = ["arablelandoflandareaag", "soil", "tropical", 'pmean', "irri_impact5" ]
tech = ["fixedtelephonesubscriptionsp", "accesstoelectricityofpopu" ]
social = ["urbanpopulationoftotalpop", "populationdensitypeoplepers_ln", "employmentinagricultureof"]
econ_poli = ["maddison_gdppc_1990_estimate_ln", "gdp", "taxrevenueofgdpgctaxt_2", "federalism_gt", "healthexpenditurepublicof", "generalgovernmentfinalconsump_ln", "legor_so"]
devel = ["slave_descendants_ext", "adj_index", "pln_sxhr_mean" ]

print ("Variance explained by each latent variable in PCA: ", pca.explained_variance_ratio_ )
print(new_df[:10])

Variance explained by each latent variable in PCA:  [0.68038003]
[[ 1.90936839]
 [ 1.40563804]
 [-0.4566214 ]
 [ 1.90728154]
 [-0.87057333]
 [-0.97274495]
 [ 0.84196337]
 [ 3.01792671]
 [-0.59944941]
 [ 2.65003383]]


In [187]:
# normalize x


dff = df2.fillna(df2.mean())
x_norm_g = StandardScaler().fit_transform(dff.loc[:, geo])
x_norm_s = StandardScaler().fit_transform(dff.loc[:, social])
x_norm_t = StandardScaler().fit_transform(dff.loc[:, tech])


pca_g = PCA(n_components=1).fit(x_norm_g)
pca_s = PCA(n_components=1).fit(x_norm_s)
pca_t = PCA(n_components=1).fit(x_norm_t)
#new_df = pcag.transform(x_norm)
var_exp = pca_g.explained_variance_ratio_

#important variables: gdp, federalism_gt, origtime2, adj_index, 

#duplicate variables

# try pca -- imagine theory suggests you look for a 1d latent representation for x2,x3,x7
dim = 1
geo = ['eleva', 'avg_temp', 'frstdays', "lp_lat_abst_fill"]
agro = ["arablelandoflandareaag", "soil", "tropical", 'pmean', "irri_impact5" ]
tech = ["fixedtelephonesubscriptionsp", "accesstoelectricityofpopu" ]
social = ["urbanpopulationoftotalpop", "populationdensitypeoplepers_ln", "employmentinagricultureof"]
econ_poli = ["maddison_gdppc_1990_estimate_ln", "gdp", "taxrevenueofgdpgctaxt_2", "federalism_gt", "healthexpenditurepublicof", "generalgovernmentfinalconsump_ln", "legor_so"]
devel = ["slave_descendants_ext", "adj_index", "pln_sxhr_mean" ]

#print ("Variance explained by each latent variable in PCA: ", pca_g.explained_variance_ratio_ )
#print(new_df[:10])


temp1 = dff.loc[:, geo].values
pcag = PCA(n_components=1)

temp2 = dff.loc[:, agro]
pcaa = PCA(n_components=1)

temp3 = dff.loc[:, tech]
pcat = PCA(n_components=1)

temp4 = dff.loc[:, social]
pcas = PCA(n_components=1)

temp5 = dff.loc[:, econ_poli]
pcaep = PCA(n_components=1)

temp6 = dff.loc[:, devel]
pcad = PCA(n_components=1)


#new_df = pca.transform(x_norm)

# create 1 dimensional representation
latent_vars_1 = pcag.fit_transform(temp1)
latent_vars_3 = pcat.fit_transform(temp3)
latent_vars_4 = pcas.fit_transform(temp4)

# check to see if this is right

#print ("Variance explained by each latent variable in PCA: ", pca_g.explained_variance_ratio_, )
#print ("\n")

# create new dataframe with the latent variables from pca1
dff['pcag'] = latent_vars_1[:,0]
dff['pcat'] = latent_vars_3[:,0]
dff['pcas'] = latent_vars_4[:,0]

latent_vars_1_reshaped = latent_vars_1.reshape(-1, 1)
latent_vars_3_reshaped = latent_vars_3.reshape(-1, 1)
latent_vars_4_reshaped = latent_vars_4.reshape(-1, 1)

# Now you can concatenate correctly
new_x_norm = np.concatenate([x_norm_g, latent_vars_1_reshaped, x_norm_t, latent_vars_3_reshaped, x_norm_s, latent_vars_4_reshaped], axis=1)

# add the latent variables to x_norm
#x_norm = np.append(x_norm_g,latent_vars,1)

#new_x_norm = np.append([x_norm_g, x_norm_t, x_norm_s],[latent_vars_1, latent_vars_3, latent_vars_4],1)

# just an example of running a model with SOME of the columns
IVs = ['pcat', 'pcag', "pcas"]

# create train / test split using dataframe
x_train, x_test, y_train, y_test = train_test_split(dff.loc[:, IVs], dff.loc[:, 'gini_disp_ext'], test_size=0.25, random_state=13)

# make sure results make sense
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)
print("-------")


# try both linear and polynomial of different degrees
linear_model = LinearRegression(normalize=True)
p2_model = LinearRegression(normalize=True)
p3_model = LinearRegression(normalize=True)

# create polynomial features
p2_features = PolynomialFeatures(degree=2)
p2_train = p2_features.fit_transform(x_train)
p2_test = p2_features.fit_transform(x_test)

p3_features = PolynomialFeatures(degree=3)
p3_train = p3_features.fit_transform(x_train)
p3_test = p3_features.fit_transform(x_test)

# now do estimation of models
lin_1 = linear_model.fit(x_train, y_train)
p2_1 = p2_model.fit(p2_train, y_train)
p3_1 = p3_model.fit(p3_train, y_train)

# predict values for test sets
lin1_predict = lin_1.predict(x_test)
p2_predict = p2_1.predict(p2_test)
p3_predict = p3_1.predict(p3_test)

print(mean_squared_error(lin1_predict, y_test))
print(mean_squared_error(p2_predict, y_test))
print(mean_squared_error(p3_predict, y_test))

# use k-fold with regularization
# remember: alpha here is lambda in most treatments
from sklearn.linear_model import LassoCV

#Lasso Cross validation
lasso_1 = LassoCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 10], random_state=0).fit(x_train, y_train)

# results
print(lasso_1.score(x_train, y_train))
print(lasso_1.score(x_test, y_test))
print("----------------")


# or you can do by hand
lambdas = (.1, .5, 1, 2.5, 5, 7.5, 10, 20, 50, 100, 200)

for i in lambdas:    
    lasso_reg = Lasso(alpha = i, max_iter=10000)
    lasso1 = lasso_reg.fit(p3_train, y_train)
    lasso1_predict = lasso1.predict(p3_test)
    print (mean_squared_error(y_test, lasso1_predict)**(.5))

(144, 3) (144,)
(48, 3) (48,)
-------
32.4768471151155
34.04617645128852
68.42488539917132
0.20590073916127083
0.302925152439919
----------------
8.2524394574363
8.175399776170545
8.081258124281891
7.813151698130076
7.523619663478078
7.317871983304903
7.1319918124477075
6.661359618365148
6.872011316801067
6.989670652580338
7.1145669055148115


In [210]:
#getting equations P1, P2, P3

#p1
feature_names = dff.loc[:, IVs].columns.tolist()

# Print linear regression equation
linear_coefficients = lin_1.coef_
linear_intercept = lin_1.intercept_
linear_equation = f"y = {linear_intercept:.4f} + " + " + ".join([f"{coef:.4f}*{feature_names[i]}" for i, coef in enumerate(linear_coefficients)])
print("Linear Regression Equation:")
print(linear_equation)




Linear Regression Equation:
y = 40.0022 + 0.1277*pcat + 0.0007*pcag + -0.0093*pcas


In [211]:
feature_names

['pcat', 'pcag', 'pcas']

In [212]:
# Print polynomial regression equations
def print_polynomial_equation(coefficients, intercept, feature_names, degree):
    terms = []
    for i, coef in enumerate(coefficients):
        if i > 0:  # Skip the intercept
            # Use the feature names for polynomial features
            terms.append(f"{coef:.4f}*{feature_names[i]}")  # Use feature_names directly without adjusting index
    equation = f"y = {intercept:.4f} + " + " + ".join(terms)
    print(f"Polynomial Regression (Degree {degree}) Equation:")
    print(equation)

# For Polynomial Regression (Degree 2)
p2_features = PolynomialFeatures(degree=2)
p2_train = p2_features.fit_transform(x_train)  # Fit to the training data
p2_feature_names = p2_features.get_feature_names(feature_names)  # Get new feature names

print_polynomial_equation(p2_coefficients, p2_intercept, p2_feature_names, 2)

# For Polynomial Regression (Degree 3)
p3_features = PolynomialFeatures(degree=3)
p3_train = p3_features.fit_transform(x_train)  # Fit to the training data
p3_feature_names = p3_features.get_feature_names(feature_names)  # Get new feature names

print_polynomial_equation(p3_coefficients, p3_intercept, p3_feature_names, 3)


Polynomial Regression (Degree 2) Equation:
y = 42.4119 + 0.2453*pcat + 0.0009*pcag + -0.0549*pcas + -0.0011*pcat^2 + 0.0002*pcat pcag + -0.0048*pcat pcas + 0.0000*pcag^2 + -0.0002*pcag pcas + 0.0025*pcas^2
Polynomial Regression (Degree 3) Equation:
y = 43.4706 + 0.3138*pcat + -0.0010*pcag + -0.1052*pcas + -0.0020*pcat^2 + 0.0003*pcat pcag + -0.0041*pcat pcas + 0.0000*pcag^2 + -0.0004*pcag pcas + 0.0015*pcas^2 + -0.0000*pcat^3 + 0.0000*pcat^2 pcag + 0.0001*pcat^2 pcas + -0.0000*pcat pcag^2 + -0.0000*pcat pcag pcas + -0.0001*pcat pcas^2 + -0.0000*pcag^3 + 0.0000*pcag^2 pcas + 0.0000*pcag pcas^2 + -0.0000*pcas^3


In [213]:
# For Linear Regression
lin_predictions = lin_1.predict(x_test)
lin_mse = mean_squared_error(y_test, lin_predictions)
print(f"Linear Regression MSE: {lin_mse:.4f}")

# For Polynomial Regression (Degree 2)
p2_predictions = p2_model.predict(p2_features.transform(x_test))  # Transform x_test for polynomial features
p2_mse = mean_squared_error(y_test, p2_predictions)
print(f"Polynomial Regression (Degree 2) MSE: {p2_mse:.4f}")

# For Polynomial Regression (Degree 3)
p3_predictions = p3_model.predict(p3_features.transform(x_test))  # Transform x_test for polynomial features
p3_mse = mean_squared_error(y_test, p3_predictions)
print(f"Polynomial Regression (Degree 3) MSE: {p3_mse:.4f}")


Linear Regression MSE: 32.4768
Polynomial Regression (Degree 2) MSE: 34.0462
Polynomial Regression (Degree 3) MSE: 68.4249
